# Overview

Projek ini bertujuan untuk mengklasifikasikan warna buah apel. Terdapat 3 class warna yang coba untuk diprediksi, yaitu hijau, merah, dan kuning. Seluruh rangkaian proses training menggunakan Google Colab dengan runtime type: "T4 - GPU"

> Dataset didapatkan dari pengumpulan secara mandiri dengan proses download gambar yang ada di internet.


> Proses labeling data dilakukan pada platform: https://www.makesense.ai/

Created by: **Taufiq Ahmadi**; Date: **January 06th 2025**

# Ekstrak Dataset

In [ ]:
!pip install rarfile


In [ ]:
import os

# Buat direktori untuk file yang diekstrak
output_dir = "/content/extracted_files"
os.makedirs(output_dir, exist_ok=True)

# Ekstrak file .rar
!unrar x "dataset_apel.rar" {output_dir}


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from dataset_apel.rar

Creating    /content/extracted_files/dataset apel                     OK
Creating    /content/extracted_files/dataset apel/images              OK
Extracting  /content/extracted_files/dataset apel/images/1710565653_65f5291502253_rxjQ5ylWgmorSRYQCbxm.webp       1%  OK 
Extracting  /content/extracted_files/dataset apel/images/211895_ilustrasi-buah-apel_665_374.webp       1%  OK 
Extracting  /content/extracted_files/dataset apel/images/3-colored-apple-green-apple-red-apple-and-yellow-apple-fruit-wallpaper-preview.jpg       2%  OK 
Extracting  /content/extracted_files/dataset apel/images/apel-buah-apel.jpg       2%  OK 
Extracting  /content/extracted_files/dataset apel/images/apel.jpg          2%  OK 
Extracting  /content/extracted_files/dataset apel/images/download (1).jpeg       3%  OK 
Extracting  /content/extracted_files/data

# Pre-process Dataset

In [ ]:
import os
import shutil
import random

name_file = []

img_dir = '/content/extracted_files/dataset apel/images'
labels_dir = '/content/extracted_files/dataset apel/labels'

train_img = '/content/dataset/train/images/'
test_img = '/content/dataset/test/images/'

train_labels = '/content/dataset/train/labels/'
test_labels = '/content/dataset/test/labels/'

composition_train = 0.6

Pada kode di bawah ini, dilakukan proses pemisahan atau pembagian antara data train dan data test. Hal ini dilakukan karena dataset yang sudah saya buat, hanya menjadi satu folder, dan belum dilakukan pemisahan.

Pemisahan data train dan data test dilakukan dengan proporsi 60/40. 60% untuk data train dan 40% untuk data test.

In [ ]:
# Buat folder target jika belum ada
os.makedirs(train_img, exist_ok=True)
os.makedirs(test_img, exist_ok=True)
os.makedirs(train_labels, exist_ok=True)
os.makedirs(test_labels, exist_ok=True)

# Ambil daftar file gambar dan label
img_files = sorted(os.listdir(img_dir))
label_files = sorted(os.listdir(labels_dir))

img_label_pairs = [(img, label) for img, label in zip(img_files, label_files) if os.path.splitext(img)[0] == os.path.splitext(label)[0]]

# Acak pasangan gambar dan label
random.shuffle(img_label_pairs)

# Hitung jumlah file untuk training
total_files = len(img_label_pairs)
num_train = int(total_files * composition_train)

# Bagi file menjadi train dan test
train_pairs = img_label_pairs[:num_train]
test_pairs = img_label_pairs[num_train:]

# Fungsi untuk menyalin file ke folder target
def copy_files(pairs, source_img_dir, source_label_dir, target_img_dir, target_label_dir):
    for img_file, label_file in pairs:
        img_src_path = os.path.join(source_img_dir, img_file)
        label_src_path = os.path.join(source_label_dir, label_file)
        img_target_path = os.path.join(target_img_dir, img_file)
        label_target_path = os.path.join(target_label_dir, label_file)

        if os.path.isfile(img_src_path) and os.path.isfile(label_src_path):
            shutil.copy(img_src_path, img_target_path)
            shutil.copy(label_src_path, label_target_path)

# Salin file ke folder train dan test
copy_files(train_pairs, img_dir, labels_dir, train_img, train_labels)
copy_files(test_pairs, img_dir, labels_dir, test_img, test_labels)

# Output hasil
print(f"Training images: {len(train_pairs)}")
print(f"Test images: {len(test_pairs)}")


Training images: 38
Test images: 26


# Modeling

In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 40.8 MB/s eta 0:00:00


## Setup Config

In [ ]:
config_content = """
path: /content/dataset
train: train/images
val: test/images
test: test/images

names:
  0: green
  1: red
  2: yellow
"""

# Simpan ke file dataset.yaml
with open('/content/dataset.yaml', 'w') as f:
    f.write(config_content)


## Train

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # Pilih model YOLOv8
model.train(data='/content/dataset.yaml', epochs=50, imgsz=640, batch=16)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 297MB/s]


Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

100%|██████████| 755k/755k [00:00<00:00, 91.1MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all         26        274      0.695       0.69      0.707      0.482



50 epochs completed in 0.027 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.61it/s]


                   all         26        274      0.695      0.752       0.72      0.496
                 green         23        131      0.848      0.779      0.844      0.574
                   red         22        111      0.757      0.784      0.816      0.568
                yellow         13         32      0.481      0.694        0.5      0.347
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7bb3cab25510>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

## Trial Predicting

In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
import os

model_path = "/content/runs/detect/train/weights/best.pt"
test_image_dir = "/content/dataset/test/images/"
output_dir = "/content/test_predictions/"

# Load model
model = YOLO(model_path)

# Warna untuk masing-masing kelas (RGB)
class_colors = {
    0: (0, 255, 0),   # Hijau untuk 'green'
    1: (255, 0, 0),   # Biru untuk 'red'
    2: (0, 0, 255)    # Merah untuk 'yellow'
}

# Prediksi gambar di direktori test
results = model.predict(
    source=test_image_dir,
    save=True,
    save_txt=True,
    conf=0.5,  # Confidence threshold
    project=output_dir,
    name="visualization_results",
    imgsz=640
)

# Fungsi untuk menampilkan gambar dengan bounding box
def visualize_results(image_path, result):
    # Load gambar
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Iterasi melalui semua prediksi
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        confidence = box.conf[0]
        cls = int(box.cls[0])

        # Teks label (class name + confidence)
        label = f"{model.names[cls]} {confidence:.2f}"

        # Tentukan warna dasar
        color = class_colors.get(cls, (255, 255, 255))

        # Draw bounding box
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Tampilkan gambar
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis("off")
    plt.show()

# Tampilkan visualisasi untuk setiap gambar di test set
for result in results:
    visualize_results(result.path, result)


Output hidden; open in https://colab.research.google.com to view.

Kode di bawah ini memproses prediksi dengan melanjutkan ke tahap pengklasifikasian pada proses memasukkan gambar hasil prediksi ke folder yang klasifikasinya sesuai.

In [ ]:
import cv2
import os
from ultralytics import YOLO

# Path ke model hasil training dan direktori gambar test
model_path = "/content/runs/detect/train/weights/best.pt"  # Model YOLOv8
test_image_dir = "/content/dataset/test/images/"           # Direktori gambar test
output_dir = "/content/classified_predictions/"            # Direktori output berdasarkan kelas

# Load model
model = YOLO(model_path)

# Buat folder output jika belum ada
os.makedirs(output_dir, exist_ok=True)

# Prediksi gambar di direktori test
results = model.predict(
    source=test_image_dir,
    save=False,   # Tidak menyimpan gambar prediksi default
    conf=0.5,     # Confidence threshold
    imgsz=640
)

# Fungsi untuk memindahkan gambar ke folder sesuai kelas
def save_by_class(image_path, result, output_dir):
    # Load gambar asli
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Iterasi setiap prediksi dalam gambar
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cls = int(box.cls[0])
        class_name = model.names[cls]

        # Buat folder kelas jika belum ada
        class_folder = os.path.join(output_dir, class_name)
        os.makedirs(class_folder, exist_ok=True)

        # Draw bounding box pada gambar
        color = (0, 255, 0) if cls == 0 else (255, 0, 0) if cls == 1 else (0, 0, 255)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)  # Bounding box
        cv2.putText(img, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Simpan gambar di folder kelas
        base_name = os.path.basename(image_path)
        output_path = os.path.join(class_folder, base_name)
        cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

# Iterasi melalui hasil prediksi
for result in results:
    save_by_class(result.path, result, output_dir)

print(f"Predicted images have been saved to {output_dir}.")



image 1/26 /content/dataset/test/images/1710565653_65f5291502253_rxjQ5ylWgmorSRYQCbxm.webp: 448x640 2 greens, 3 reds, 14.9ms
image 2/26 /content/dataset/test/images/Manfaat-Apel-Hijau.png: 416x640 2 yellows, 11.2ms
image 3/26 /content/dataset/test/images/OIP (11).jpeg: 448x640 2 greens, 2 reds, 10.9ms
image 4/26 /content/dataset/test/images/OIP (12).jpeg: 512x640 23 greens, 18 reds, 2 yellows, 9.5ms
image 5/26 /content/dataset/test/images/OIP (16).jpeg: 384x640 6 yellows, 8.9ms
image 6/26 /content/dataset/test/images/OIP (18).jpeg: 384x640 2 greens, 6 reds, 7.7ms
image 7/26 /content/dataset/test/images/OIP (4).jpeg: 416x640 1 green, 2 reds, 8.5ms
image 8/26 /content/dataset/test/images/OIP (5).jpeg: 544x640 1 green, 1 red, 1 yellow, 14.8ms
image 9/26 /content/dataset/test/images/OIP (6).jpeg: 512x640 1 green, 1 red, 11.8ms
image 10/26 /content/dataset/test/images/OIP (7).jpeg: 512x640 4 reds, 10.6ms
image 11/26 /content/dataset/test/images/OIP (8).jpeg: 448x640 1 green, 1 red, 9.4ms
i

Sama dengan kode sebelumnya, tetapi kode di bawah ini melakukan proses cropping terhadap gambar yang berhasil diklasifikasikan atau diprediksikan ke suatu class. Setelah proses pemotongan, selanjutnya akan dimasukkan pada folder yang sesuai dengan class-nya.

In [ ]:
import cv2
import os
from ultralytics import YOLO

# Path ke model hasil training dan direktori gambar test
model_path = "/content/runs/detect/train/weights/best.pt"
test_image_dir = "/content/dataset/test/images/"
output_dir = "/content/cropped_predictions/"

# Load model
model = YOLO(model_path)

# Buat folder untuk menyimpan hasil crop per class
class_dirs = {}
class_names = ["green", "red", "yellow"]  # Nama kelas
for class_name in class_names:
    class_dir = os.path.join(output_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)
    class_dirs[class_name] = class_dir

# Prediksi gambar di direktori test
results = model.predict(
    source=test_image_dir,
    save=False,
    conf=0.5,  # Confidence threshold
    imgsz=640
)

# Proses setiap gambar
for result in results:
    # Load gambar asli
    img = cv2.imread(result.path)

    # Iterasi melalui semua bounding box
    for i, box in enumerate(result.boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cls = int(box.cls[0])
        class_name = class_names[cls]
        conf = box.conf[0]

        # Potong gambar berdasarkan bounding box
        cropped_img = img[y1:y2, x1:x2]

        # Simpan gambar yang dipotong ke folder class
        save_path = os.path.join(class_dirs[class_name], f"{class_name}_{i}_{os.path.basename(result.path)}")
        cv2.imwrite(save_path, cropped_img)

        print(f"Cropped and saved: {save_path}")



image 1/26 /content/dataset/test/images/1710565653_65f5291502253_rxjQ5ylWgmorSRYQCbxm.webp: 448x640 2 greens, 3 reds, 10.1ms
image 2/26 /content/dataset/test/images/Manfaat-Apel-Hijau.png: 416x640 2 yellows, 7.4ms
image 3/26 /content/dataset/test/images/OIP (11).jpeg: 448x640 2 greens, 2 reds, 7.0ms
image 4/26 /content/dataset/test/images/OIP (12).jpeg: 512x640 23 greens, 18 reds, 2 yellows, 7.6ms
image 5/26 /content/dataset/test/images/OIP (16).jpeg: 384x640 6 yellows, 7.5ms
image 6/26 /content/dataset/test/images/OIP (18).jpeg: 384x640 2 greens, 6 reds, 6.8ms
image 7/26 /content/dataset/test/images/OIP (4).jpeg: 416x640 1 green, 2 reds, 7.3ms
image 8/26 /content/dataset/test/images/OIP (5).jpeg: 544x640 1 green, 1 red, 1 yellow, 7.8ms
image 9/26 /content/dataset/test/images/OIP (6).jpeg: 512x640 1 green, 1 red, 6.9ms
image 10/26 /content/dataset/test/images/OIP (7).jpeg: 512x640 4 reds, 6.1ms
image 11/26 /content/dataset/test/images/OIP (8).jpeg: 448x640 1 green, 1 red, 7.1ms
image 

# Generate requirements.txt for listing all library

In [ ]:
!pip freeze

In [ ]:
!pip freeze > requirements.txt